In [86]:
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import dataframe_image as dfi
import pylatex as pl
from pylatex import Command, Document, Section, Subsection
from pylatex.utils import NoEscape, italic
from mdutils.mdutils import *
from mdutils import *
import os

In [87]:
wheels_tab=['Brand','Model','Portrait','Analysis']
brands=os.listdir('trips')
tab_size=0
for brand in brands:
    wheels=os.listdir('trips/'+brand)
    tab_size+=len(wheels)
    for wheel in wheels:
        wheels_tab+=[brand,wheel,"""<img src="imgs/wheels/portrait/"""+wheel+""".webp" alt="drawing" width="64"/>""" ,"[details](analysis/"+wheel+".md)"]
tab_size

6

In [112]:
envs=[]
consos=[]
summaries=[]
for root, dirs, files in os.walk('stats'):
    for file in files:
        if file == 'env.csv':
            envs+=[root+"/"+file]
        if file == 'conso.csv':
            consos+=[root+"/"+file]
        if file == 'summary.csv':
            summaries+=[root+"/"+file]

envs=[pd.read_csv(env) for env in envs]
consos=[pd.read_csv(conso) for conso in consos]
summaries=[pd.read_csv(summary) for summary in summaries]

envs = pd.concat(envs)
consos = pd.concat(consos)
summaries = pd.concat(summaries)

summaries=summaries[summaries['stat']=='mean'].groupby('stat').mean()
consos['foo']=1
consos=consos.groupby('foo').mean()
envs['foo']=1
envs=envs[['trip_distance_km','rider_weight_kg', 'wheel_weight_kg','global_weight_kg', 'tire_pressure_bars','outdoor_temperature_c','foo']].groupby('foo').mean()
avg_dist=envs['trip_distance_km'].values[0]
avg_rider_weight=envs['rider_weight_kg'].values[0]
avg_wheel_weight=envs['wheel_weight_kg'].values[0]
avg_total_weight=envs['global_weight_kg'].values[0]
avg_speed=summaries['speed'].values[0]
avg_power=summaries['power'].values[0]
avg_conso=consos['conso_corrected_kwh'].values[0]
avg_whkm=consos['average_wh/km'].values[0]

avg_data=['**Field**','**Unit**','**Data**','Average Trip Distance','km',round(avg_dist,2),'Average Rider Weight','kg',round(avg_rider_weight,2),'Average Wheel Weight','kg',round(avg_wheel_weight,2),'Average Speed','km.h',round(avg_speed,2),'Average Power','Watts',round(avg_power,2),'Average Consumption per trip','kWh',round(avg_conso,2),'Average Consumption','Wh.km',round(avg_whkm,2)]


In [113]:
envs

,trip_distance_km,rider_weight_kg,wheel_weight_kg,global_weight_kg,tire_pressure_bars,outdoor_temperature_c
foo,,,,,,
1,25.872455,85.909091,27.4,113.309091,2.445455,19.090909


In [114]:
consos

,trip_id,initial_battery_kwh,end_batt_lvl_pct,end_batt_voltage,estimated_end_batt_lvl_pct,conso_kwh,regen_kwh,conso_corrected_kwh,average_wh/km,voltage_sag_coeff
foo,,,,,,,,,,
1,0.727273,1.719727,56.0,86.399091,58.818182,0.765455,-0.035455,0.73,23.677273,-0.380182


In [115]:
summaries

,trip_id,wh.km-1,speed,power,current,current_phase,temperature,motor_temperature,battery_temperature,safety_margin
stat,,,,,,,,,,
mean,0.727273,23.678218,26.618605,659.879355,7.419672,9.708976,33.011822,60.306916,31.027822,62.57882


In [116]:
mdFile = MdUtils(file_name="README.md",title='')
# Additional Markdown syntax...

mdFile.new_header(level=1, title="EUCS_testbench")
mdFile.write("**EUCs logs analyzed to provide real statistics**")
mdFile.new_line()
mdFile.write("> This page aims at providing insight of different EUC performance in real life usage.")


mdFile.new_header(level=2, title="Methods")
mdFile.write("- **Each analysis consists of the analyze of a single trip's data for a single wheel.**")
mdFile.new_line()
mdFile.new_line()
mdFile.write("- So far data must come from **WheelLog** or **EUC World**")

mdFile.new_header(level=2, title="Average data based on all available data")
mdFile.new_table(columns=3, rows=8, text=avg_data, text_align='center')

mdFile.new_header(level=2, title="Existing data")
mdFile.new_table(columns=4, rows=tab_size+1, text=wheels_tab, text_align='center')

mdFile.new_header(level=2, title="Further data")

mdFile.write("The more data for different wheels is collected, the more acurate analysis can be.")
mdFile.write("For more acurate information, many trips should be analyzed and results aggregated to provide better insights of real world usafe of those devices.")
mdFile.write("---")
mdFile.create_md_file()